# Data loading ...

In [1]:
import pandas as pd

#saved_x_tuesday.csv

df = pd.read_csv('flatten_data/saved_x_saturday.csv',index_col=None)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.head(5)


,mean_uAx,mean_uAy,mean_uAz,mean_uRx,mean_uRy,mean_uRz,sd_uAx,sd_uAy,sd_uAz,sd_uRx,...,sum_uRz,energy_uAx,energy_uAy,energy_uAz,fft_uAx,fft_uAy,fft_uAz,fft_uRx,fft_uRy,fft_uRz
0,-2.220446e-17,-3.330669e-17,-1.110223e-17,2.220446e-17,8.326673e-17,0.000000e+00,1.0,1.0,1.0,1.0,...,0.000000e+00,7.395571e-32,1.664004e-31,1.848893e-32,-4.440892e-16,-1.776357e-15,-4.440892e-16,2.220446e-16,2.775558e-15,8.881784e-16
1,0.000000e+00,-2.220446e-17,4.440892e-17,-2.775558e-17,-1.110223e-17,-4.440892e-17,1.0,1.0,1.0,1.0,...,-1.776357e-15,0.000000e+00,7.395571e-32,2.958228e-31,8.326673e-16,-8.881784e-16,2.220446e-15,-8.881784e-16,-9.992007e-16,-1.998401e-15
2,-3.330669e-17,-4.440892e-17,1.471046e-16,5.551115e-18,1.665335e-17,0.000000e+00,1.0,1.0,1.0,1.0,...,0.000000e+00,1.664004e-31,2.958228e-31,3.245962e-30,-1.554312e-15,-1.443290e-15,6.106227e-15,4.440892e-16,1.554312e-15,-1.110223e-16
3,0.000000e+00,2.220446e-17,3.774758e-16,-3.330669e-17,2.775558e-17,4.440892e-17,1.0,1.0,1.0,1.0,...,1.776357e-15,0.000000e+00,7.395571e-32,2.137320e-29,1.110223e-16,0.000000e+00,1.332268e-14,-4.440892e-16,8.881784e-16,2.220446e-15
4,1.110223e-17,0.000000e+00,4.440892e-17,2.220446e-17,-1.665335e-17,1.110223e-17,1.0,1.0,1.0,1.0,...,4.440892e-16,1.848893e-32,0.000000e+00,2.958228e-31,4.440892e-16,-8.881784e-16,1.110223e-15,2.220446e-16,-3.330669e-16,9.992007e-16


In [2]:
y = pd.read_csv('flatten_data/saved_y_saturday.csv',index_col=None)
y = y.loc[:, ~y.columns.str.contains('^Unnamed')]
y.head(5)

,action_label
0,0
1,0
2,0
3,0
4,0


# Overlay for MLPv5 with any enhancement

In [3]:
from pynq import Overlay
from pynq import allocate
import pynq.lib.dma
import numpy as np

INPUT_SIZE = 69
OUTPUT_SIZE = 9

class Model:
    def __init__(self, bitfile, paramfile):
        # Initialize Overlay & DMA
        self.overlay = Overlay(bitfile)
        self.dma = self.overlay.axi_dma_0
        
        # Load weights and bias
        f = open(paramfile, "r")
        self.params = f.read().split(',')
        for i in range(len(self.params)):
            self.params[i] = np.float32(self.params[i])
        self.numofparams = len(self.params)
        
        # Initialize DMA buffer
        self.input_buffer = allocate(shape=(self.numofparams+INPUT_SIZE,), dtype=np.float32)
        for i in range (self.numofparams):
            self.input_buffer[i] = self.params[i]
        self.res = allocate(shape=(2*OUTPUT_SIZE,), dtype=np.float32)
        
    def classify(self, in_x):
        for i in range(INPUT_SIZE):
            self.input_buffer[i+self.numofparams] = in_x[i]

        self.dma.sendchannel.transfer(self.input_buffer)
        self.dma.recvchannel.transfer(self.res)
        self.dma.sendchannel.wait()
        self.dma.recvchannel.wait()
        max_index = np.argmax(self.res)
        return max_index, self.res[max_index]
        

## FPGA version with pipelining enhancement

In [4]:
import time

mlp = Model("bitstreams/mlpv5_1.bit", "mlpv5.csv")

correct = 0
total = 0
start_time = time.time()
for i in range(df.values.shape[0]):
    input_x = df.values[i]
    pred, prob = mlp.classify(input_x)
#     print(pred,prob, y.values[i])
    if (pred == y.values[i]):
        correct += 1
    total += 1
print("-----%s ms in average elapsed for each transfer-----" %(1000*(time.time()-start_time)/total))
print("TOTAL ACCURACY: ", correct/float(total))

-----0.852348600613455 ms in average elapsed for each transfer-----
TOTAL ACCURACY:  0.9266880048773053


# MLPv5 on CPU version (without framework)

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt 
import h5py as h5

W01 = 69
W12 = 128
W23 = 64
W34 = 9

# Take in params
# Read H5 file
f = h5.File("weights_files/features_fit_transform_model_weights.h5", "r")
# Get and print list of datasets within the H5 file
datasetNames = list(f.keys())

for k1 in f:
    print(k1)
    for k2 in f[k1].keys():
        print(k2)
        for k3 in f[k1][k2].keys():  
            print(k3)

b1 = f['dense_12']['dense_12']['bias:0'][()]
b2 = f['dense_13']['dense_13']['bias:0'][()]
b3 = f['dense_14']['dense_14']['bias:0'][()]
w1 = f['dense_12']['dense_12']['kernel:0'][()]
w2 = f['dense_13']['dense_13']['kernel:0'][()]
w3 = f['dense_14']['dense_14']['kernel:0'][()]
w3

dense_12
dense_12
bias:0
kernel:0
dense_13
dense_13
bias:0
kernel:0
dense_14
dense_14
bias:0
kernel:0


array([[-1.97719097e-01,  9.17642862e-02,  1.50109544e-01,
         1.69360593e-01, -8.68740976e-02,  1.60182580e-01,
        -2.53711734e-03, -4.43915308e-01,  8.58310908e-02],
       [ 2.67669439e-01, -3.06111187e-01,  1.53358176e-01,
         2.73639768e-01,  6.49441108e-02, -9.26099867e-02,
        -2.58671463e-01, -5.44847250e-01,  1.09471187e-01],
       [ 1.29793540e-01,  6.91684941e-03,  2.06769168e-01,
        -4.93505448e-01, -2.70051539e-01, -6.64841413e-01,
         3.20975721e-01,  3.31431240e-01,  2.35518202e-01],
       [-6.74625188e-02, -3.33491303e-02, -1.28625914e-01,
         8.59580114e-02,  1.09870300e-01, -6.85273290e-01,
        -5.60524203e-02,  1.66440502e-01, -3.26543190e-02],
       [-5.30548505e-02, -1.02214448e-01,  3.35262269e-02,
         4.85509224e-02,  3.72087322e-02, -5.21878242e-01,
        -2.28882089e-01,  1.67884573e-01,  2.57191509e-02],
       [-8.52659225e-01,  1.33825734e-01, -2.82511562e-01,
        -1.19362943e-01, -5.81621565e-02,  3.662616

In [22]:
def relu(X):
    return np.maximum(0,X)

def softmax(X):
    expo = np.exp(X)
    expo_sum = np.sum(np.exp(X))
    return expo/expo_sum

def dense_layer(x, w, b):
    temp = np.matmul(x,w)
    o = temp + b
    return relu(o)

def output_layer(x, w, b):
    temp = np.matmul(x,w)
    o = temp + b
    return softmax(o)


In [23]:
import time
x = np.ones((1,W01), dtype='f4')
correct = 0
total = 0
start_time = time.time()
for k in range(df.values.shape[0]):
#     if y[k] != 0:
#         continue
    for i in range(W01):
        x[0][i] = (df.values[k][i])
        x[0][i] = np.float32(x[0][i])

    out1 = dense_layer(x, w1, b1)
    out2 = dense_layer(out1, w2, b2)
    pred = output_layer(out2, w3, b3)
    
    res_index = np.argmax(pred)
#     print(res_index, y.values[k])
#     print(pred[0][res_index])
    if (np.argmax(pred) == y.values[k]):
        correct += 1
    total +=  1
print("-----%s ms in average elapsed for each transfer-----" %(1000*(time.time()-start_time)/total))
print("TOTAL ACCURACY: ", correct/total)


-----5.564084741295592 ms in average elapsed for each transfer-----
TOTAL ACCURACY:  0.9266880048773053
